In [ ]:
import os
import subprocess
import psutil
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm                         # ← add tqdm import
import mlflow
import mlflow.pytorch
from pynvml import (
    nvmlInit, nvmlDeviceGetHandleByIndex,
    nvmlDeviceGetUtilizationRates, nvmlDeviceGetMemoryInfo,
    nvmlDeviceGetTemperature, NVML_TEMPERATURE_GPU
)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

mlflow.set_experiment("PannsMLP")
try:
    mlflow.end_run()
except:
    pass
mlflow.start_run(log_system_metrics=True)

# log GPU info
gpu_info = next(
    (subprocess.run(cmd, capture_output=True, text=True).stdout
        for cmd in ["nvidia-smi", "rocm-smi"]
        if subprocess.run(f"command -v {cmd}", shell=True,
                          capture_output=True).returncode == 0),
    "No GPU found."
)
mlflow.log_text(gpu_info, "gpu-info.txt")

nvmlInit()
gpu_handle = nvmlDeviceGetHandleByIndex(0)

def log_system_metrics_mlflow(step=None):
    mlflow.log_metric("system.cpu.utilization", psutil.cpu_percent(), step=step)
    mem = psutil.virtual_memory()
    mlflow.log_metric("system.memory.used", mem.used, step=step)
    mlflow.log_metric("system.memory.percent", mem.percent, step=step)
    gpu_util = nvmlDeviceGetUtilizationRates(gpu_handle).gpu
    mlflow.log_metric("system.gpu.0.utilization", gpu_util, step=step)
    gpu_mem = nvmlDeviceGetMemoryInfo(gpu_handle)
    mlflow.log_metric("system.gpu.0.memory.used", gpu_mem.used, step=step)
    mlflow.log_metric("system.gpu.0.memory.percent",
                      (gpu_mem.used / gpu_mem.total) * 100, step=step)
    gpu_temp = nvmlDeviceGetTemperature(gpu_handle, NVML_TEMPERATURE_GPU)
    mlflow.log_metric("system.gpu.0.temperature", gpu_temp, step=step)

In [ ]:
class EmbeddingDataset(Dataset):
    def __init__(self, manifest_csv: str, metadata_csv: str, embeddings_key="embedding"):
        """
        manifest_csv: path to manifest_train.csv or manifest_test.csv
        metadata_csv: path to full train.csv (for secondary_labels)
        """
        m_df = pd.read_csv(manifest_csv)
        meta = pd.read_csv(metadata_csv, usecols=["filename", "secondary_labels"])
        meta["recording_id"] = meta.filename.str.replace(r"\.ogg$", "", regex=True)
        meta["secondary_list"] = meta.secondary_labels.fillna("").str.split()
        sec_map = dict(zip(meta.recording_id, meta.secondary_list))

        self.rows = []
        all_labels = set()
        # wrap iteration in tqdm to show loading progress
        for _, row in tqdm(m_df.iterrows(),
                            desc=f"Building EmbeddingDataset ({os.path.basename(manifest_csv)})",
                            total=len(m_df)):
            rid = row.chunk_id.split("_chk")[0]
            prim = row.primary_label
            secs = sec_map.get(rid, [])
            labels = [prim] + secs
            all_labels.update(labels)
            self.rows.append({
                "emb_path": row.emb_path,
                "labels": labels
            })

        self.classes = sorted(all_labels)
        self.label2idx = {lab: i for i, lab in enumerate(self.classes)}
        self.num_classes = len(self.classes)
        self.embeddings_key = embeddings_key

    def __len__(self):
        return len(self.rows)

    def __getitem__(self, idx):
        r = self.rows[idx]
        arr = np.load(r["emb_path"])[self.embeddings_key]  # (n_windows, emb_dim)
        x = arr.mean(axis=0).astype(np.float32)            # (emb_dim,)
        y = np.zeros(self.num_classes, dtype=np.float32)
        for lab in r["labels"]:
            y[self.label2idx[lab]] = 1.0
        return x, y

In [ ]:
class EmbeddingClassifier(nn.Module):
    def __init__(self, emb_dim, num_classes, hidden_dims=[512,256], dropout=0.3):
        super().__init__()
        self.fc1 = nn.Linear(emb_dim, hidden_dims[0])
        self.bn1 = nn.BatchNorm1d(hidden_dims[0])
        self.drop1 = nn.Dropout(dropout)
        self.fc2 = nn.Linear(hidden_dims[0], hidden_dims[1])
        self.bn2 = nn.BatchNorm1d(hidden_dims[1])
        self.drop2 = nn.Dropout(dropout)
        self.out = nn.Linear(hidden_dims[1], num_classes)

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.drop1(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.drop2(x)
        return self.out(x)

In [ ]:
TRAIN_MANIFEST = "/home/jovyan/Features/manifest_train.csv"
TEST_MANIFEST  = "/home/jovyan/Features/manifest_test.csv"
TRAIN_CSV      = "/home/jovyan/Data/birdclef-2025/train.csv"

BATCH_SIZE  = 32
LR          = 1e-3
EPOCHS      = 20
HIDDEN_DIMS = [512, 256]
DROPOUT     = 0.3

train_ds = EmbeddingDataset(TRAIN_MANIFEST, TRAIN_CSV)
test_ds  = EmbeddingDataset(TEST_MANIFEST,  TRAIN_CSV)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,  num_workers=4)
test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

emb_dim = np.load(train_ds.rows[0]["emb_path"])[train_ds.embeddings_key].shape[1]
model   = EmbeddingClassifier(emb_dim, train_ds.num_classes, HIDDEN_DIMS, DROPOUT).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

mlflow.log_params({
    "input_dim": emb_dim,
    "hidden_dims": HIDDEN_DIMS,
    "dropout": DROPOUT,
    "batch_size": BATCH_SIZE,
    "lr": LR,
    "epochs": EPOCHS
})

In [ ]:
best_test_acc = 0.0
best_ckpt     = None

for epoch in range(1, EPOCHS + 1):
    # — Train —
    model.train()
    train_bar = tqdm(train_loader, desc=f"Epoch {epoch} Train", unit="batch")
    running_loss, correct, total = 0.0, 0, 0

    for xb, yb in train_bar:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        logits = model(xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * xb.size(0)
        preds = (torch.sigmoid(logits) > 0.5).float()
        correct += (preds == yb).all(dim=1).sum().item()
        total   += xb.size(0)

        train_bar.set_postfix({
            "batch_loss": f"{loss.item():.4f}",
            "acc":         f"{correct/total:.4f}"
        })

    train_loss = running_loss / total
    train_acc  = correct / total

    # — Test —
    model.eval()
    test_bar = tqdm(test_loader, desc=f"Epoch {epoch} Test ", unit="batch")
    test_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for xb, yb in test_bar:
            xb, yb = xb.to(device), yb.to(device)
            logits = model(xb)
            loss = criterion(logits, yb)

            test_loss += loss.item() * xb.size(0)
            preds = (torch.sigmoid(logits) > 0.5).float()
            correct += (preds == yb).all(dim=1).sum().item()
            total   += xb.size(0)

            test_bar.set_postfix({
                "batch_loss": f"{loss.item():.4f}",
                "acc":         f"{correct/total:.4f}"
            })

    test_loss /= total
    test_acc   = correct / total

    # checkpoint
    ckpt = f"ckpt_epoch_{epoch}.pt"
    torch.save({
        "epoch": epoch,
        "model_state": model.state_dict(),
        "optim_state": optimizer.state_dict(),
        "train_loss": train_loss,
        "test_loss": test_loss
    }, ckpt)

    # MLflow logging
    mlflow.log_metrics({
        "train_loss": train_loss,
        "train_accuracy": train_acc,
        "test_loss": test_loss,
        "test_accuracy": test_acc
    }, step=epoch)
    log_system_metrics_mlflow(step=epoch)
    mlflow.log_artifact(ckpt, artifact_path="checkpoints")

    if test_acc > best_test_acc:
        best_test_acc = test_acc
        best_ckpt     = ckpt

    print(f"→ Epoch {epoch}/{EPOCHS}  "
          f"Train loss={train_loss:.4f}, acc={train_acc:.4f} │ "
          f"Test loss={test_loss:.4f}, acc={test_acc:.4f}")

In [ ]:
mlflow.log_metric("best_test_accuracy", best_test_acc)
mlflow.log_artifact(best_ckpt, artifact_path="model")
mlflow.pytorch.log_model(model, "panns_mlp_model")
mlflow.end_run()